# Structured Data: NumPy's Structured Arrays

While often our data can be well represented by a homogeneous array of values, sometimes this is not the case. This section demosntrates the use of NumPy's structured arrays and record arrays, which provide efficient storage for compound, heterogeneous data. While hte patterns shown here are useful for siple operations, scenarios like this often len themselves to the use of Pandas `Dataframe`.

In [1]:
import numpy as np

Imagine that we have several categories of data on a number of people (say, name, age, and weight), and we'd like to store these values for use in a Python program. It would be possible to store these in three separate arrays:

In [2]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

But this is a bit clumsy. There's nothing here that tells us that the three arrays are related; it would be more natural if we could use a single structure to store all of this data. NumPy can handle htis through structured arrays, which are arrays with compound data types.

Recall that previoysly we created a simple array using an expression like this:

In [3]:
x = np.zeros(4, dtype=int)

 We can similarly create a structured array using a compound data type specification:

In [4]:
# Use a compound data type for structured arrays
data = np.zeros(4, dtype={'names':('name', 'age', 'weight'),
                          'formats':('U10', 'i4', 'f8')})
print(data.dtype)                          

[('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


Here `U10` translates to "Unicode string of maximum length 10", `i4` translates to "4-byte integer", and `f8` translates to "8-byte float". We'll discuss other options for these types codes in the following section.

Now that we've created an empty container array, we can fill the array with our lists of values:

In [5]:
data['name'] = name
data['age'] = age
data['weight'] = weight 
print(data)

[('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]


As we had hoped, the data is now arranged together in one convenient block of memory.

The handy thing with structured arrays is that you can now refer to values either by index or by name:

In [7]:
data['name']

array(['Alice', 'Bob', 'Cathy', 'Doug'], dtype='<U10')

Get the name from the last row

In [14]:
data[-1]['name']

'Doug'

Using Boolean masking, this even allows you to do some more sophisticated operations such as filtering on age:

In [17]:
# Get names where age is under 30
data[data['age'] < 30]['name']

array(['Alice', 'Doug'], dtype='<U10')

Note that if you'd like to do any operations that are any more complicated than these, you should probably consider the Pandas package. Pandas provide a `Dataframe` object, which is a structure built on NumPy arrays that offers a variaty of useful data manipulation functionality similar to what we've shown here, as well as much, much more.

## Creating Structured Arrays

Structured array data types can be specified in a number of ways. Earlier, we saw the dictionary method:

In [21]:
np.dtype({'names':('name', 'age', 'weight'), 'formats': ('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

For clarity, numerical types can be specified using Python types or NumPy `dtype` instead:

In [ ]:
np.dtype({'names':('name', 'age', 'weight'), 'formats': ((np.str_, 10), int, np.float32)})

A compound type can also be specified as a list of tuples:

In [ ]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

The shortened string format codes may seem confusin, but they are built on simple principles. The first (optional) characters is `<` or `>`, which means "little endian" or "big endian" and specifies the ordering convenction for significant bits.

The next character specifies the type of data: characters, bytes, ints, floating points, and so on. The last character or characters represents the size of the object in bytes.

```
Character 	Description 	Example
'b' 	Byte 	np.dtype('b')
'i' 	Signed integer 	np.dtype('i4') == np.int32
'u' 	Unsigned integer 	np.dtype('u1') == np.uint8
'f' 	Floating point 	np.dtype('f8') == np.int64
'c' 	Complex floating point 	np.dtype('c16') == np.complex128
'S', 'a' 	String 	np.dtype('S5')
'U' 	Unicode string 	np.dtype('U') == np.str_
'V' 	Raw data (void) 	np.dtype('V') == np.void

```

# More Advanced Compound Types

It is possible to define even more advanced compund types. For example, you can create a type where each element contains an array or matrix of values. Here, we'll create a data type with a `mat` component consisting of a *3 x 3* floating-point matrix:

In [26]:
tp = np.dtype([('id', 'i8'), ('mat', 'f8', (3, 3))])
X = np.zeros(3, dtype=tp)
print(X[0])
print(X['mat'][0])

(0, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Now each element in the `x` array consist of an `id` and a *3 x 3* matrix. Why would you use this rather than a simple multidimensional array, or perhaps a Python dictionary? The reason is that this NumPy `dtype` direclty maps onto a C structure definiton, so the buffer containing the array content can be accessed directly within an appropriately written C program. If you find yourself writing a Pythin inferface to a legacy C or Fortran library that manipulates structured data, you'll probably find structured arrays quite useful!

# RecordArrays: Structured Arrays with a Twist

NumPy also provides the `np.recarray` class, which is almost identical to the structured darrays just described, but with one additional feature: fields can be accessed as attributes rather than as dictionary keys. Recall that we previously accessed the ages by writing:

In [29]:
data['age']

array([25, 45, 37, 19])

If we view our data as a record array instead, we can access this with slightly fewer keystrokes:

In [30]:
data_rec = data.view(np.recarray)
data_rec.age

array([25, 45, 37, 19])

The down side is that for record arrays, there is some extra overhead involved in accessing the fields, even when using the same syntax. We can see this here:

In [33]:
%timeit data['age']
%timeit data_rec['age']
%timeit data_rec.age

197 ns ± 59.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
3.24 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.4 µs ± 609 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Where the more convenient notation is worth the addicitonal overhead will depend on your own application

## On to Pandas

This section on structured and record arrays is purposely at the end of this chapter,
becasue it leads so well into the next package we will cover: Pandas. Structured array like the ones discussed here are good to know about for certain situations, especially in case you're using NumPy arrays to map onto binary data formats in C, Fortran, or another language. For day-to-day use of structured data, the Pandas package is a much better choice, and we'll dive into a full discussion of it in the chapter that follows.